In [11]:
# List of vertices (6 vertices)
vertices = ['v0', 'v1', 'v2', 'v3', 'v4', 'v5']  

# Define edges between the vertices
edges = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (0, 3)]

# Initialize a matrix to hold the rigidity values
R = Matrix(SR, len(edges), len(vertices))  

# Dictionary to store edge variables
edge_vars = {}

# Function to build the rigidity matrix
def buildMatrix(vertices, edges, R, edge_vars):
    for k, (i, j) in enumerate(edges):  # Loop over each edge (i, j)
        # Create the symbolic variable name for edge (i, j)
        edge_label = 'v{}{}'.format(i, j)  # Example: v01, v12, etc.
        
        # Ensure the variable is declared in the edge_vars dictionary
        edge_vars[edge_label] = var(edge_label)  # Create a symbolic variable
        
        R[k, i] = edge_vars[edge_label]      # Place 'vij' (the variable) in the matrix
        R[k, j] = -edge_vars[edge_label]  # Place '-vij' (the variable) in the opposite vertex column
    
    return R

# Build the rigidity matrix and store the edge variables
R = buildMatrix(vertices, edges, R, edge_vars)
print("Rigidity matrix in terms of edge vectors:")
show(R)

# Calculate R * R_transpose
RRt = R * R.transpose()
#print("R * R^T in terms of the dot products:")
#show(RRt)

# Function to convert each entry into variables
def convert_entry_to_variable(entry):
    entry_str = str(entry)  # Convert to string to handle it easily

    # Initialize coefficient as 1
    coeff = 1
    negative_sign = False

    # Check for negative signs and coefficients
    if entry_str.startswith('-'):
        negative_sign = True
        entry_str = entry_str[1:]  # Remove the negative sign for processing

    # Handle coefficients if present
    if entry_str[0].isdigit():
        parts = entry_str.split('*')
        coeff = int(parts[0])  # Extract coefficient
        entry_str = '*'.join(parts[1:])  # Join remaining parts after removing coefficient

    # Handle zero entries
    if entry_str == '0':
        return 0  # Return 0 directly for zero entries

    # Handle the case for squared terms (e.g., 2*v12^2)
    if '^2' in entry_str:
        base = entry_str.replace('^2', '')  # Remove the '^2'
        new_var_name = f"{base}{base[1:]}"  # Duplicate the number part (e.g., v12 -> v1212)
        final_var = f"v{new_var_name[1:]}"  # Ensure it starts with 'v' and remove leading 'v'
        return var(final_var) * coeff * (-1 if negative_sign else 1)  # Include coefficient and handle sign

    # Handle the case for product terms (e.g., v12*v23)
    elif '*' in entry_str:
        factors = entry_str.split('*')  # Split on '*'
        new_var_name = ''.join(factor[1:] for factor in factors)  # Concatenate without 'v'
        final_var = f"v{new_var_name}"  # Add 'v' prefix
        return var(final_var) * coeff * (-1 if negative_sign else 1)  # Include coefficient and handle sign

    # Handle single variables (e.g., v12)
    else:
        new_var_name = entry_str  # Keep it as is (it already starts with 'v')
        if new_var_name:  # Ensure the new_var_name is not empty
            return var(new_var_name) * coeff * (-1 if negative_sign else 1)  # Include coefficient and handle sign
        else:
            raise ValueError("Variable name is empty; cannot create a variable.")

# Replace each entry in RRt with the new variable
for i in range(RRt.nrows()):
    for j in range(RRt.ncols()):
        entry = RRt[i, j]
        try:
            RRt[i, j] = convert_entry_to_variable(entry)  # Replace each entry with the new variable
        except ValueError:
            pass  # Silently ignore the error without printing anything


# Print the resulting matrix RRt
print("R * R^T in terms of edge variables:")
show(RRt)


Rigidity matrix in terms of edge vectors:


[ v01 -v01    0    0    0    0]
[ v02    0 -v02    0    0    0]
[   0  v12 -v12    0    0    0]
[   0  v13    0 -v13    0    0]
[   0    0  v23 -v23    0    0]
[ v03    0    0 -v03    0    0]

R * R^T in terms of edge variables:


[2*v0101   v0102  -v0112  -v0113       0   v0103]
[  v0102 2*v0202   v0212       0  -v0223   v0203]
[ -v0112   v0212 2*v1212   v1213  -v1223       0]
[ -v0113       0   v1213 2*v1313   v1323   v0313]
[      0  -v0223  -v1223   v1323 2*v2323   v0323]
[  v0103   v0203       0   v0313   v0323 2*v0303]

In [12]:
show(RRt.det())

-v0112^2*v0223^2*v0313^2 + 2*v0112*v0113*v0212*v0223*v0313*v0323 - v0113^2*v0212^2*v0323^2 + 4*v0113^2*v0223^2*v0303*v1212 + 4*v0103*v0113*v0223^2*v0313*v1212 + 4*v0101*v0223^2*v0313^2*v1212 + 4*v0113^2*v0203*v0223*v0323*v1212 + 4*v0102*v0113*v0223*v0313*v0323*v1212 + 4*v0113^2*v0202*v0323^2*v1212 - 4*v0112*v0113*v0223^2*v0303*v1213 - 2*v0103*v0112*v0223^2*v0313*v1213 - 4*v0112*v0113*v0203*v0223*v0323*v1213 - 2*v0103*v0113*v0212*v0223*v0323*v1213 - 2*v0102*v0112*v0223*v0313*v0323*v1213 - 4*v0101*v0212*v0223*v0313*v0323*v1213 - 4*v0112*v0113*v0202*v0323^2*v1213 - 2*v0102*v0113*v0212*v0323^2*v1213 - v0103^2*v0223^2*v1213^2 + 4*v0101*v0223^2*v0303*v1213^2 - 2*v0102*v0103*v0223*v0323*v1213^2 + 4*v0101*v0203*v0223*v0323*v1213^2 - v0102^2*v0323^2*v1213^2 + 4*v0101*v0202*v0323^2*v1213^2 - 4*v0113^2*v0212*v0223*v0303*v1223 - 2*v0112*v0113*v0203*v0223*v0313*v1223 - 4*v0103*v0113*v0212*v0223*v0313*v1223 - 2*v0102*v0112*v0223*v0313^2*v1223 - 4*v0101*v0212*v0223*v0313^2*v1223 - 2*v0113^2*v0203*v0212*v0323*v1223 - 4*v0112*v0113*v0202*v0313*v0323*v1223 - 2*v0102*v0113*v0212*v0313*v0323*v1223 + 2*v0103*v0113*v0203*v0223*v1213*v1223 - 4*v0102*v0113*v0223*v0303*v1213*v1223 - 2*v0102*v0103*v0223*v0313*v1213*v1223 + 4*v0101*v0203*v0223*v0313*v1213*v1223 + 4*v0103*v0113*v0202*v0323*v1213*v1223 - 2*v0102*v0113*v0203*v0323*v1213*v1223 - 2*v0102^2*v0313*v0323*v1213*v1223 + 8*v0101*v0202*v0313*v0323*v1213*v1223 - v0113^2*v0203^2*v1223^2 + 4*v0113^2*v0202*v0303*v1223^2 + 4*v0103*v0113*v0202*v0313*v1223^2 - 2*v0102*v0113*v0203*v0313*v1223^2 - v0102^2*v0313^2*v1223^2 + 4*v0101*v0202*v0313^2*v1223^2 + 4*v0112^2*v0223^2*v0303*v1313 + 4*v0112^2*v0203*v0223*v0323*v1313 + 4*v0103*v0112*v0212*v0223*v0323*v1313 + 4*v0112^2*v0202*v0323^2*v1313 + 4*v0102*v0112*v0212*v0323^2*v1313 + 4*v0101*v0212^2*v0323^2*v1313 + 4*v0103^2*v0223^2*v1212*v1313 - 16*v0101*v0223^2*v0303*v1212*v1313 + 8*v0102*v0103*v0223*v0323*v1212*v1313 - 16*v0101*v0203*v0223*v0323*v1212*v1313 + 4*v0102^2*v0323^2*v1212*v1313 - 16*v0101*v0202*v0323^2*v1212*v1313 - 4*v0103*v0112*v0203*v0223*v1223*v1313 - 4*v0103^2*v0212*v0223*v1223*v1313 + 8*v0102*v0112*v0223*v0303*v1223*v1313 + 16*v0101*v0212*v0223*v0303*v1223*v1313 - 8*v0103*v0112*v0202*v0323*v1223*v1313 + 4*v0102*v0112*v0203*v0323*v1223*v1313 - 4*v0102*v0103*v0212*v0323*v1223*v1313 + 8*v0101*v0203*v0212*v0323*v1223*v1313 + 4*v0103^2*v0202*v1223^2*v1313 - 4*v0102*v0103*v0203*v1223^2*v1313 + 4*v0101*v0203^2*v1223^2*v1313 + 4*v0102^2*v0303*v1223^2*v1313 - 16*v0101*v0202*v0303*v1223^2*v1313 - 4*v0112*v0113*v0212*v0223*v0303*v1323 - 2*v0112^2*v0203*v0223*v0313*v1323 - 2*v0103*v0112*v0212*v0223*v0313*v1323 - 2*v0112*v0113*v0203*v0212*v0323*v1323 - 2*v0103*v0113*v0212^2*v0323*v1323 - 4*v0112^2*v0202*v0313*v0323*v1323 - 4*v0102*v0112*v0212*v0313*v0323*v1323 - 4*v0101*v0212^2*v0313*v0323*v1323 + 4*v0103*v0113*v0203*v0223*v1212*v1323 - 8*v0102*v0113*v0223*v0303*v1212*v1323 - 4*v0102*v0103*v0223*v0313*v1212*v1323 + 8*v0101*v0203*v0223*v0313*v1212*v1323 + 8*v0103*v0113*v0202*v0323*v1212*v1323 - 4*v0102*v0113*v0203*v0323*v1212*v1323 - 4*v0102^2*v0313*v0323*v1212*v1323 + 16*v0101*v0202*v0313*v0323*v1212*v1323 - 2*v0103*v0112*v0203*v0223*v1213*v1323 - 2*v0103^2*v0212*v0223*v1213*v1323 + 4*v0102*v0112*v0223*v0303*v1213*v1323 + 8*v0101*v0212*v0223*v0303*v1213*v1323 - 4*v0103*v0112*v0202*v0323*v1213*v1323 + 2*v0102*v0112*v0203*v0323*v1213*v1323 - 2*v0102*v0103*v0212*v0323*v1213*v1323 + 4*v0101*v0203*v0212*v0323*v1213*v1323 - 2*v0112*v0113*v0203^2*v1223*v1323 - 2*v0103*v0113*v0203*v0212*v1223*v1323 + 8*v0112*v0113*v0202*v0303*v1223*v1323 + 4*v0102*v0113*v0212*v0303*v1223*v1323 + 4*v0103*v0112*v0202*v0313*v1223*v1323 - 2*v0102*v0112*v0203*v0313*v1223*v1323 + 2*v0102*v0103*v0212*v0313*v1223*v1323 - 4*v0101*v0203*v0212*v0313*v1223*v1323 + 4*v0103^2*v0202*v1213*v1223*v1323 - 4*v0102*v0103*v0203*v1213*v1223*v1323 + 4*v0101*v0203^2*v1213*v1223*v1323 + 4*v0102^2*v0303*v1213*v1223*v1323 - 16*v0101*v0202*v0303*v1213*v1223*v1323 - v0112^2*v0203^2*v1323^2 - 2*v0103*v0112*v020